# 00 - Kaggle Environment Setup (GPU)

This notebook sets up the Kaggle GPU environment for the Format Matters project.

**Prerequisites:**
- Kaggle account with GPU quota
- GPU accelerator enabled in notebook settings

**Steps:**
1. Enable GPU in Kaggle settings
2. Install additional dependencies
3. Verify GPU availability
4. Check system information
5. Set up project directories

## ⚙️ Enable GPU

**IMPORTANT**: Before running this notebook:
1. Click the **Settings** icon (⚙️) in the top-right
2. Under **Accelerator**, select **GPU**
3. Click **Save**

The notebook will restart with GPU enabled.

## 1. Install Additional Dependencies

Kaggle comes with PyTorch pre-installed. We only need to install format-specific libraries.

In [ ]:
%%time
# Install format libraries and utilities
!pip install -q webdataset==0.2.86 tfrecord==1.15 lmdb==1.4.1 pyarrow==16.1.0 psutil==5.9.8

print("\n✓ Additional dependencies installed successfully!")

## 2. Verify GPU Availability

In [ ]:
!nvidia-smi

In [ ]:
import torch

print("GPU Configuration:")
print(f"  CUDA available: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"  CUDA version: {torch.version.cuda}")
    print(f"  cuDNN version: {torch.backends.cudnn.version()}")
    print(f"  GPU count: {torch.cuda.device_count()}")
    print(f"  GPU name: {torch.cuda.get_device_name(0)}")
    print(f"  GPU memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")
    print("\n✓ GPU is available and ready!")
else:
    print("\n⚠ WARNING: GPU not available!")
    print("  Please enable GPU in notebook settings (⚙️ → Accelerator → GPU)")

## 3. Verify Core Dependencies

In [ ]:
import importlib
import sys

# Required packages
required_packages = [
    'torch',
    'torchvision',
    'numpy',
    'pandas',
    'PIL',  # Pillow
    'webdataset',
    'tfrecord',
    'lmdb',
    'pyarrow',
    'psutil',
    'matplotlib',
    'tqdm',
]

print("Checking installed packages:\n")
all_ok = True

for package_name in required_packages:
    try:
        module = importlib.import_module(package_name)
        version = getattr(module, '__version__', 'unknown')
        print(f"✓ {package_name:15s} {version}")
    except ImportError as e:
        print(f"✗ {package_name:15s} NOT INSTALLED")
        all_ok = False

print("\n" + "="*60)
if all_ok:
    print("✓ All dependencies available!")
else:
    print("⚠ Some dependencies missing - re-run installation cell")

## 4. System Information

In [ ]:
import platform
import psutil
import os

print("System Information:")
print(f"  OS: {platform.system()} {platform.release()}")
print(f"  Platform: {platform.platform()}")
print(f"  Processor: {platform.processor()}")
print(f"  CPU cores: {psutil.cpu_count(logical=False)} physical, {psutil.cpu_count(logical=True)} logical")
print(f"  RAM: {psutil.virtual_memory().total / (1024**3):.1f} GB")
print(f"  Python: {platform.python_version()}")
print(f"\nWorking directory: {os.getcwd()}")
print(f"Available space: {psutil.disk_usage('/kaggle/working').free / (1024**3):.1f} GB")

## 5. Kaggle Directory Structure

Kaggle has specific directories:
- `/kaggle/input/` - Read-only datasets
- `/kaggle/working/` - Read-write workspace (saved with notebook)
- `/kaggle/tmp/` - Temporary storage (not saved)

In [ ]:
from pathlib import Path

print("Kaggle directories:\n")

# Check input datasets
input_dir = Path('/kaggle/input')
if input_dir.exists():
    datasets = list(input_dir.iterdir())
    print(f"Input datasets ({len(datasets)}):")
    for ds in datasets[:10]:  # Show first 10
        print(f"  - {ds.name}")
    if len(datasets) > 10:
        print(f"  ... and {len(datasets) - 10} more")
else:
    print("No input datasets found")

print(f"\nWorking directory: /kaggle/working")
print(f"Temporary directory: /kaggle/tmp")

## 6. Create Project Directories

In [ ]:
from pathlib import Path

# Use /kaggle/working as base
base_dir = Path('/kaggle/working/format-matters')

# Define directory structure
directories = [
    base_dir / 'data/raw/cifar10',
    base_dir / 'data/raw/imagenet-mini',
    base_dir / 'data/built',
    base_dir / 'runs',
    base_dir / 'notebooks',
    base_dir / 'scripts',
]

print("Creating project directories:\n")
for dir_path in directories:
    dir_path.mkdir(parents=True, exist_ok=True)
    print(f"  ✓ {dir_path}")

print("\n✓ Directory structure created successfully!")
print(f"\nProject root: {base_dir}")

## 7. Test Basic Functionality

In [ ]:
import torch
import torchvision
import numpy as np
import pandas as pd
from PIL import Image

print("Testing basic functionality:\n")

# Test PyTorch with GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
x = torch.randn(2, 3, 224, 224, device=device)
print(f"✓ PyTorch tensor on {device}: {x.shape}")

# Test torchvision transforms
transform = torchvision.transforms.Compose([
    torchvision.transforms.Resize(224),
    torchvision.transforms.ToTensor(),
])
print(f"✓ Torchvision transforms ready")

# Test numpy
arr = np.random.rand(10, 10)
print(f"✓ NumPy array creation: {arr.shape}")

# Test pandas
df = pd.DataFrame({'a': [1, 2, 3], 'b': [4, 5, 6]})
print(f"✓ Pandas DataFrame: {df.shape}")

# Test PIL
img = Image.new('RGB', (100, 100), color='red')
print(f"✓ PIL Image creation: {img.size}")

# Test GPU computation
if torch.cuda.is_available():
    a = torch.randn(1000, 1000, device='cuda')
    b = torch.randn(1000, 1000, device='cuda')
    c = torch.matmul(a, b)
    print(f"✓ GPU matrix multiplication: {c.shape}")

print("\n✓ All basic functionality tests passed!")

## 8. Environment Summary

In [ ]:
import json
from datetime import datetime

env_info = {
    "timestamp": datetime.now().isoformat(),
    "environment": "kaggle",
    "python_version": sys.version,
    "pytorch_version": torch.__version__,
    "cuda_available": torch.cuda.is_available(),
    "cuda_version": torch.version.cuda if torch.cuda.is_available() else None,
    "gpu_name": torch.cuda.get_device_name(0) if torch.cuda.is_available() else None,
    "gpu_memory_gb": round(torch.cuda.get_device_properties(0).total_memory / (1024**3), 1) if torch.cuda.is_available() else None,
    "os": platform.system(),
    "cpu_count": psutil.cpu_count(logical=True),
    "ram_gb": round(psutil.virtual_memory().total / (1024**3), 1),
    "disk_free_gb": round(psutil.disk_usage('/kaggle/working').free / (1024**3), 1),
}

print("Environment Summary:")
print(json.dumps(env_info, indent=2))

# Save to file
env_file = Path('/kaggle/working/format-matters/runs/env_kaggle.json')
env_file.parent.mkdir(parents=True, exist_ok=True)
env_file.write_text(json.dumps(env_info, indent=2))
print(f"\n✓ Environment info saved to: {env_file}")

## 9. Dataset Setup Instructions

To add datasets to Kaggle:

### Option 1: Use Kaggle Datasets
1. Find or upload your dataset to Kaggle Datasets
2. In this notebook, click **Add Data** (right sidebar)
3. Search for and add your dataset
4. It will appear under `/kaggle/input/<dataset-name>/`

### Option 2: Upload Directly
1. Click **Add Data** → **Upload**
2. Upload your dataset files
3. They will appear under `/kaggle/input/`

### Recommended Datasets
- **CIFAR-10**: Will be auto-downloaded in `01_prepare_datasets.ipynb`
- **ImageNet-mini** or **Tiny-ImageNet-200**: Upload as Kaggle dataset

### Copy to Working Directory
In `01_prepare_datasets.ipynb`, we'll copy from `/kaggle/input/` to `/kaggle/working/format-matters/data/raw/` so builders can write to `data/built/`.

## ✅ Setup Complete!

Your Kaggle GPU environment is ready. Next steps:

1. **Add datasets**: Use "Add Data" to include ImageNet-mini or Tiny-ImageNet-200
2. **Prepare datasets**: Run `01_prepare_datasets.ipynb`
3. **Build formats**: Run builder notebooks (02-05)
4. **Run experiments**: Execute training notebooks (20-21)
5. **Analyze results**: Use analysis notebooks (30-31)
6. **Download results**: Use `scripts/zip_runs_for_download.ipynb`

**Important Notes:**
- Kaggle notebooks have a **9-hour runtime limit**
- GPU quota is limited - use efficiently
- Save important results frequently
- Use the zip utility to download results before session ends